In [1]:
import numpy as np
import igl
import meshplot as mp
import ipywidgets as iw
import time
import scipy.sparse as sp
import triangle as tr
import matplotlib.pyplot as plt

In [2]:
v, f = igl.read_triangle_mesh('data/woody-hi.off')
cage = np.load('data/woody-hi.cage-1.npy')
v -= v.min(axis=0)
v /= v.max()

In [5]:
cage_segment = (np.vstack((np.arange(cage.shape[0]), np.arange(cage.shape[0])+1))).T
cage_segment[-1,-1] = 0

cage_dict = dict(vertices=cage[:,:2], segments=cage_segment)
cage_tri = tr.triangulate(cage_dict, 'pqa0.01')

cage_v = cage_tri['vertices']
cage_v = np.hstack((cage_v, np.zeros((cage_v.shape[0],1))))
cage_f = cage_tri['triangles']
cage_markers = cage_tri['vertex_markers']

In [6]:
num_cage_boundary_v = cage.shape[0]

In [7]:
Lw = igl.cotmatrix(cage_v, cage_f)
M = igl.massmatrix(cage_v, cage_f, igl.MASSMATRIX_TYPE_VORONOI)
Minv = sp.diags(1 / M.diagonal())

In [ ]:
A = Minv * Lw
Aff = A[np.argwhere(cage_markers!=1)[:,0],:]
Aff = Aff[:,np.argwhere(cage_markers!=1)[:,0]]
Afc = A[np.argwhere(cage_markers!=1)[:,0],:]
Afc = Afc[:,np.argwhere(cage_markers==1)[:,0]]
xc = np.eye(np.sum(cage_markers))
rhs = -Afc * xc
xf = sp.linalg.spsolve(Aff, rhs)
x = np.zeros((cage_v.shape[0], np.sum(cage_markers)))
x[np.argwhere(cage_markers==1)[:,0],:] = xc
x[np.argwhere(cage_markers!=1)[:,0],:] = xf

In [ ]:
p = mp.plot(v, f)
p.add_points(cage, shading={"point_color":"green", "point_size":0.1})
p.add_lines(cage, np.vstack((cage[1:],cage[0])))

In [ ]:
cage_oid = p.add_mesh(cage_v, cage_f, shading={"wireframe":True})
# p.update_object(oid=cage_oid, colors=x[:,3])

In [ ]:
def within_triangle(p, v, f):
    for i in range(f.shape[0]):
        v0 = v[f[i,0]]
        v1 = v[f[i,1]]
        v2 = v[f[i,2]]
        A = np.vstack((v1-v0, v2-v0)).T
        b = (p-v0)
        sol = np.linalg.lstsq(A, b, rcond=None)[0]
        if (sol[0] >= 0 and sol[1] >= 0 and sol[0]+sol[1]<=1):
            return i

In [ ]:
def barycentric_coord(p, v, f, i):
    v0 = v[f[i,0]]
    v1 = v[f[i,1]]
    v2 = v[f[i,2]]
    A = np.vstack((v0,v1,v2)).T
    A = np.vstack((A, np.ones((1,3))))
    b = np.hstack((p, np.array([1,])))
    w = np.linalg.lstsq(A, b, rcond=None)[0]
    return w

In [ ]:
hc = np.zeros((v.shape[0], np.sum(cage_markers)))
for i in range(v.shape[0]):
    cage_tri_ind = within_triangle(v[i], cage_v, cage_f)
    w = barycentric_coord(v[i], cage_v, cage_f, cage_tri_ind)
    hc[i,:] = w[0]*x[cage_f[cage_tri_ind,0], :] \
            + w[1]*x[cage_f[cage_tri_ind,1], :] \
            + w[2]*x[cage_f[cage_tri_ind,2], :]

In [ ]:
test = hc @ cage_v[np.argwhere(cage_markers==1)[:,0],:]
p.update_object(oid=0, vertices=test)

In [ ]:
cage_v[0,1] -= 0.09
p.update_object(oid=cage_oid, vertices=cage_v)

In [ ]:
cage_v[1,1] -= 0.09
p.update_object(oid=cage_oid, vertices=cage_v)

In [ ]:
cage_v[2,1] -= 0.09
p.update_object(oid=cage_oid, vertices=cage_v)

In [ ]:
test = hc @ cage_v[np.argwhere(cage_markers==1)[:,0],:]
p.update_object(oid=0, vertices=test)

In [ ]:
p.remove_object(1)
p.add_points(cage_v[:num_cage_boundary_v], shading={"point_color":"green", "point_size":0.1})